In [ ]:
options(dplyr.summarise.inform = FALSE)
library(tidyverse)
library(data.table)
library(survival)

In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(dirname(dirname(dirname(getwd()))),'/stats.r'))

#### 0 - Read in scaled and prepared data

In [ ]:
system.time(go <- readRDS(paste0(SHARE_DIR, "ready_ex.Rds")))

In [ ]:
all <- go$data_ready
features <- go$features

#### 1 - Define Cohorts

In [ ]:
cohorts <- list()
top_cohorts <- unique(all$location)
cohorts[['pan']] <- all
for( i in unique(all$location) ) cohorts[[i]] <- all %>% filter(location == i)

#### 2 - Run Survival and BOR Analyses across cohorts

- Overall Survival

In [ ]:
os_out <- data.frame()
for( c in names(cohorts)){
  df <- cohorts[[c]]
  if( c == "pan"){ os <- scanner("Surv(Y_os_days, Y_os_event)", features, "+ as.factor(location) + clin_age + clin_sex", "df", "coxph")} 
  else { os <- scanner("Surv(Y_os_days, Y_os_event)", features, "+ clin_age + clin_sex", "df", "coxph")}
  os_out <- rbind(os_out, os %>% mutate(cohort = c ))
}

- Best response

In [ ]:
bor_out <- data.frame()
for( c in names(cohorts)){
  df <- cohorts[[c]]
  if( c == "pan"){ bor <- scanner("Y_bor", features, "+ as.factor(location) + clin_age + clin_sex", "df", "bor")} 
  else { bor <- scanner("Y_bor", features, "+ clin_age + clin_sex", "df", "bor")}
  bor_out <- rbind(bor_out, bor %>% mutate(cohort = c ))
}

- OS adjusted for response

In [ ]:
os_out_adj <- data.frame()
for( c in names(cohorts)){
  df <- cohorts[[c]]
  if( c == "pan"){ os <- scanner("Surv(Y_os_days, Y_os_event)", features, " + Y_bor + as.factor(location) + clin_age + clin_sex", "df", "coxph")} 
  else { os <- scanner("Surv(Y_os_days, Y_os_event)", features, "+ Y_bor + clin_age + clin_sex", "df", "coxph")}
  os_out_adj <- rbind(os_out_adj, os %>% mutate(cohort = c ))
}

#### 3 - Share the output 

In [ ]:
out <- rbind(os_out, bor_out, os_out_adj) %>% mutate(pval_by = p.adjust(pval, method = "BY")) 

In [ ]:
fwrite(out, paste0(UTIL_DIR, "go.csv"))